[Check out this notebook](https://www.kaggle.com/ayuraj/baseline-efficientnet-using-tf-and-w-b) which is used to train the model. I am continuously working to improve the score.

This notebook will be used to run inference and create the `sample_submission.csv` file.

In [ ]:
%%capture
!pip install ../input/keras-efficientnet-whl/Keras_Applications-1.0.8-py3-none-any.whl
!pip install ../input/keras-efficientnet-whl/efficientnet-1.1.1-py3-none-any.whl

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import *
from tensorflow.keras.models import *

import efficientnet.keras as efn

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm

In [ ]:
MODEL_PATH_ROOT = '../input/cassava-model/'
MODEL_PATHS = [MODEL_PATH_ROOT+file for file in os.listdir(MODEL_PATH_ROOT) if file.endswith('.h5')]

print(MODEL_PATHS)

In [ ]:
tf.keras.backend.clear_session()

model = tf.keras.models.load_model(MODEL_PATHS[0])
model.summary()

In [ ]:
IMAGE_SIZE = 380
BATCH_SIZE = 1
AUTOTUNE = tf.data.experimental.AUTOTUNE
CLASS_NUMS = 5

TEST_DIR = '../input/cassava-leaf-disease-classification/test_images/'
test_images = os.listdir(TEST_DIR)

In [ ]:
image_paths = [os.path.join(TEST_DIR, img_path) for img_path in test_images]

print(image_paths)

In [ ]:
def decode_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img)
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.image.resize(img, (IMAGE_SIZE,IMAGE_SIZE))
    
    return img

def get_dataloader():
    testloader = tf.data.Dataset.from_tensor_slices((image_paths)).map(decode_image, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
    
    return testloader

In [ ]:
testloader = get_dataloader()

img = next(iter(testloader))
print(img.shape)
plt.imshow(img[0]);

In [ ]:
test_size = len(image_paths)
test_preds = np.zeros((test_size, CLASS_NUMS))

for model_path in MODEL_PATHS:
    print(f'Loading model from: {model_path}')
    
    # Load trained model
    tf.keras.backend.clear_session()
    model = tf.keras.models.load_model(model_path)
    
    # Get dataloader
    testloader = get_dataloader()
    
    # Perform inference
    predictions = []
    preds = model.predict(testloader)
    predictions.extend(preds)

    test_preds += np.array(predictions) / len(MODEL_PATHS)
    
final_test_preds = np.argmax(test_preds, axis=-1)

In [ ]:
final_test_preds

In [ ]:
sub = pd.DataFrame({'image_id': test_images, 'label': final_test_preds})
display(sub)
sub.to_csv('submission.csv', index = False)